In [1]:
import warnings
import pandas as pd
import numpy as np
import implicit

from src.utils import get_score_top, get_score_model, aggregate_users, create_sparse_matrix

warnings.filterwarnings('ignore')

Будем использовать Collaborative filtering, поэтому понадобятся нам только транзакции. Посмотрим на них

In [2]:
interactions_df = pd.read_csv('interactions.csv')
interactions_df.head()

,row,col,data
0,0,3568,1.0
1,0,3827,1.0
2,0,4844,1.0
3,0,5734,1.0
4,0,6518,1.0


Делим на train и test

In [3]:
msk = np.random.rand(len(interactions_df)) < 0.8
train = interactions_df[msk]
test = interactions_df[~msk]

# Топовые элементы
В качестве базовой модели используем топовые элементы в качестве предсказаний

In [4]:
tops = train.groupby('col').count().sort_values('data', ascending=False)
tops

,row,data
col,,
8638,727,727
17955,722,722
5113,686,686
4657,646,646
10227,636,636
...,...,...
3796,1,1
15020,1,1
15027,1,1


In [5]:
top_items = list(np.array(tops.index)[:30])
top_items

[8638,
 17955,
 5113,
 4657,
 10227,
 10466,
 197,
 12469,
 8982,
 3922,
 6289,
 4361,
 10067,
 5297,
 3806,
 4634,
 2301,
 7581,
 8666,
 5645,
 3572,
 3565,
 4844,
 187,
 15892,
 9461,
 8481,
 10068,
 5562,
 1212]

In [6]:
aggregated_test = aggregate_users(test.values)
aggregated_train = aggregate_users(train.values)

In [7]:
get_score_top(aggregated_test, top_items)

100%|█████████████████████████████████| 20396/20396 [00:00<00:00, 116770.05it/s]


0.009302945166642224

Данное число важно, оно дает нам опору. Мы теперь понимаем движемся ли мы вперед

# ML методы
Для оставшихся методов нам нужно посчитать разреженную матрицу. Матрицу размером n_users * n_items. 
Где на пересечении пользователя и товара стоит 1 если он его купил и 0 иначе

In [8]:
X_sparse = create_sparse_matrix(30910, 18494, aggregated_train)

In [9]:
X_sparse.shape

(30910, 18495)

## Метод ALS 
Он использует разложение разреженной матрицы на матрицу объектов и пользователей. Далее фиксируя одну при помощи наименьших квадратов приближает другую, далее наоборот, и так несколько итераций

In [10]:
model = implicit.als.AlternatingLeastSquares(factors=64, regularization=0.0, iterations=16)
model.fit(X_sparse)

  0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
get_score_model(aggregated_test, aggregated_train, model, 18494)

100%|████████████████████████████████████| 20396/20396 [00:31<00:00, 644.61it/s]


0.023094831920377402

Как видим, мы превзошли базовый алгоритм предсказания только топовых товаров

## Метод Cosine recommender
Он предсказывает покупки пытаясь по ближайшим по косинусному расстоянию векторов K пользователям предсказать покупки целевого

In [18]:
model = implicit.nearest_neighbours.CosineRecommender(K=3)
model.fit(X_sparse)

  0%|          | 0/18495 [00:00<?, ?it/s]

In [19]:
get_score_model(aggregated_test, aggregated_train, model, 18494)

100%|███████████████████████████████████| 20396/20396 [00:03<00:00, 5128.99it/s]


0.027209355427069194

Метод превзошел и ALS и тем более базовый алгоритм